# Setup

In [1]:
import sys
# sys.path.append("/kaggle/input/python-helpers/")
sys.path.append("../")
import common_utils
from common_utils import (
    check_memory_usage, check_memory_by_global_variable, # For memory handling
    get_time_now, cprint, # For logging
    get_cols, downcast_to_32bit, my_concat, my_power, my_log, list_diff, # For preprocessing
    plot_feature_importance, plot_scatterplot, # For visualization
    get_feature_summary, # For feature engineering & feature selection
    check_auc # For EDA
)
from comp_utils import (
    zero_sum, clean_df
)

In [2]:
import gc
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
%load_ext autoreload
%autoreload

In [4]:
# This should be equivalent to '/kaggle/input' in kaggel environment
# Meaning to say, we should place our working data folder same as the Kaggle datasets structure to make our life easier
DATA_DIR = '../data/'

In [5]:
%%time
train = pd.read_csv(f"{DATA_DIR}/optiver-trading-at-the-close/train.csv")

CPU times: user 3.73 s, sys: 453 ms, total: 4.19 s
Wall time: 4.19 s


In [6]:
%%time
train.to_parquet(f'{DATA_DIR}/optiver-train-data/raw_train.parquet')

CPU times: user 1.69 s, sys: 90.3 ms, total: 1.78 s
Wall time: 1.63 s


# Simple Cleaning
Steps:
- Remove columns
- Downcast DataType
- Rename columns

Further Enhancements:
- To 16bit? Maybe too much for now, memory still ok, and only int16 is possible without affecting data quality
- Not sure if we could use row_id & time_id to ease our preprocessing

In [7]:
def clean_df(df, columns_to_drop=['row_id', 'time_id']):
    df = df.drop(columns=columns_to_drop, errors="ignore")
    df = downcast_to_32bit(df)
    df = df.rename(
        columns={
            "seconds_in_bucket": "seconds",
            "imbalance_size": "imb_size",
            "imbalance_buy_sell_flag": "imb_flag",
            "reference_price": "ref_price",
            "wap": "wa_price", 
        }
    )
    return df

In [8]:
print(train.shape)
train.head()

(5237980, 17)


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [9]:
%%time
train = clean_df(train)

Before downcast: 	RAM used: 2.9 GB


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 175.60it/s]

After downcast: 	RAM used: 2.6 GB


CPU times: user 111 ms, sys: 207 ms, total: 318 ms
Wall time: 317 ms


In [10]:
print(train.shape)
train.head()

(5237980, 15)


,stock_id,date_id,seconds,imb_size,imb_flag,ref_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wa_price,target
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,60651.500000,1.000026,8493.030273,1.0,-3.029704
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.040039,1.000660,20605.089844,1.0,-5.519986
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,37956.000000,1.000298,18995.000000,1.0,-8.389950
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,2324.899902,1.000214,479032.406250,1.0,-4.010201
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849


- Simple cleaning is done successfully :)

In [11]:
%%time
train.to_parquet(f'{DATA_DIR}/optiver-train-data/cleaned_train.parquet')

CPU times: user 1.21 s, sys: 71.4 ms, total: 1.28 s
Wall time: 1.13 s


# Simple Clipping

Steps:
- Clip price columns from both tails
- Clip volume columns from upper tail
- Clip target column from both tails
- Create 2 extra binary target columns

Further Enhancements:
- To implement stock-based clipping instead of global clipping, because some stock might have high trading volumes / higher volatility
- By defining the bound using outlier factor instead of simple percentile for all [price / volume] columns

In [12]:
PRICE_CLIP_PERCENTILE = 0.01 # 1 - (2 * PRICE_CLIPPER_TAIL / 100) will remained unclip
VOLUME_CLIPPER_UPPER_TAIL = 0.01 # only the top VOLUME_CLIPPER_UPPER_TAIL proportion will be clipped

In [13]:
MIN_TARGET = -100 # Target lower than -100 will be clip to -100
MAX_TARGET = 100 # Target higher than 100 will be clip to 100

In [14]:
MILD_TARGET_LOWER_BOUND = -4.5
MILD_TARGET_UPPER_BOUND = 4.5

In [15]:
def get_price_clippers(df, price_cols, price_clip_percentile=PRICE_CLIP_PERCENTILE):
    price_clippers = {}
    for price_col in price_cols:
        upper_bound = np.percentile(df[price_col].dropna(), 100 - price_clip_percentile)
        lower_bound = np.percentile(df[price_col].dropna(), price_clip_percentile)
        price_clippers[price_col] = (lower_bound, upper_bound)
        cprint(f"For {price_col}, the global clip bound is", end=" ", color="blue")
        cprint(f"({lower_bound:.4f}, {upper_bound:.4f})", color="green")
    return price_clippers

In [16]:
base_price_cols = get_cols(train, contains="price")
price_clippers = get_price_clippers(train, base_price_cols)

For ref_price, the global clip bound is (0.9798, 1.0246)
For far_price, the global clip bound is (0.7866, 1.3063)
For near_price, the global clip bound is (0.8942, 1.1092)
For bid_price, the global clip bound is (0.9793, 1.0223)
For ask_price, the global clip bound is (0.9814, 1.0249)
For wa_price, the global clip bound is (0.9802, 1.0233)


In [17]:
def get_volume_clippers(df, volume_cols, volume_clip_upper_percentile=VOLUME_CLIPPER_UPPER_TAIL):
    volume_clippers = {}
    for volume_col in volume_cols:
        upper_bound = int(round(np.percentile(df[volume_col].dropna(), 100 - volume_clip_upper_percentile), -3))
        volume_clippers[volume_col] = (0, upper_bound)
        cprint(f"For {volume_col}, the global clip bound is", end=" ", color="blue")
        cprint(f"(0, {upper_bound:,.0f})", color="green")
    return volume_clippers

In [18]:
base_volume_cols = get_cols(train, contains="size")
volume_clippers = get_volume_clippers(train, base_volume_cols)

For imb_size, the global clip bound is (0, 587,223,000)
For matched_size, the global clip bound is (0, 4,502,560,000)
For bid_size, the global clip bound is (0, 2,392,000)
For ask_size, the global clip bound is (0, 2,755,000)


In [19]:
def clip_df(df, price_clippers=None, volume_clippers=None):
    # Find the column list for each group
    price_cols = get_cols(df, endswith="price")
    volume_cols = get_cols(df, endswith="size")
    flag_cols = get_cols(df, endswith="flag")
    base_cols = price_cols + volume_cols + flag_cols
    
    if price_clippers is None:
        price_clippers = get_price_clippers(df, price_cols)
    
    if volume_clippers is None:
        volume_clippers = get_volume_clippers(df, volume_cols)
    
    # Clip price columns
    for price_col in price_cols:
        df[price_col] = df[price_col].clip(*price_clippers[price_col])
        
    # Clip volume columns
    for volume_col in volume_cols:
        df[volume_col] = df[volume_col].clip(*volume_clippers[volume_col])
    
    # Clip target columns (if applicable)
    if "target" in df.columns:
        df["clipped_target"] = df["target"].clip(MIN_TARGET, MAX_TARGET)
        # This 2 transformed binary targets are for the feature analysis and potentially for supporting model(s)
        df["is_positive_target"] = (df["target"] > 0).astype(int)
        df["is_mild_target"] = df["target"].between(MILD_TARGET_LOWER_BOUND, MILD_TARGET_UPPER_BOUND).astype(int)
    return df

In [20]:
# Set the float formatting options
pd.options.display.float_format = lambda x: f"{x:,.0f}"

In [21]:
train[get_cols(train, "size")].describe()

,imb_size,matched_size,bid_size,ask_size
count,"5,237,760","5,237,760","5,237,980","5,237,980"
mean,"5,715,294","45,100,236","51,814","53,576"
std,"20,515,906","139,841,296","111,421","129,355"
min,0,"4,317",0,0
25%,"84,534","5,279,576","7,375","7,824"
50%,"1,113,604","12,882,638","21,969","23,018"
75%,"4,190,951","32,700,130","55,832","57,878"
max,"2,982,027,776","7,713,681,920","30,287,840","54,405,000"


In [22]:
%%time
train = clip_df(train, price_clippers=price_clippers, volume_clippers=volume_clippers)

CPU times: user 182 ms, sys: 144 ms, total: 326 ms
Wall time: 323 ms


In [23]:
train[get_cols(train, "size")].describe()

,imb_size,matched_size,bid_size,ask_size
count,"5,237,760","5,237,760","5,237,980","5,237,980"
mean,"5,687,389","44,978,084","51,576","53,208"
std,"18,984,529","135,028,233","94,601","98,628"
min,0,"4,317",0,0
25%,"84,534","5,279,575","7,375","7,824"
50%,"1,113,604","12,882,638","21,969","23,018"
75%,"4,190,951","32,700,130","55,832","57,878"
max,"587,223,000","4,502,560,000","2,392,000","2,755,000"


- Simple clipping is done successfully :)

In [24]:
pd.reset_option("display.float_format")

In [25]:
%%time
train.to_parquet(f'{DATA_DIR}/optiver-train-data/clipped_train.parquet')

CPU times: user 1.47 s, sys: 39.4 ms, total: 1.51 s
Wall time: 1.4 s


In [26]:
joblib.dump(price_clippers, f"{DATA_DIR}/optiver-preprocess-helpers/price_clippers.pkl")
joblib.dump(volume_clippers, f"{DATA_DIR}/optiver-preprocess-helpers/volume_clippers.pkl")

['../data//optiver-preprocess-helpers/volume_clippers.pkl']

# Reference Data Extraction 